In [1]:
import sys
import importlib
import os
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
data_dir = "../datasets/artificial_data/3d_shapes"

def create_list():
    pairs = []
    all_files = os.listdir(data_dir)
    for filename in all_files:
        label = int(filename.split("_").pop().replace(".pt", ""))
        label_dict = { "size": label }
        file_path = os.path.join(data_dir, filename)
        pairs.append({"image": file_path, "label": label_dict})
    return pairs

pairs = create_list()
print(pairs)

[{'image': '../datasets/artificial_data/3d_shapes/volume_40_label_15.pt', 'label': {'size': 15}}, {'image': '../datasets/artificial_data/3d_shapes/volume_0_label_29.pt', 'label': {'size': 29}}, {'image': '../datasets/artificial_data/3d_shapes/volume_10_label_26.pt', 'label': {'size': 26}}, {'image': '../datasets/artificial_data/3d_shapes/volume_15_label_13.pt', 'label': {'size': 13}}, {'image': '../datasets/artificial_data/3d_shapes/volume_39_label_15.pt', 'label': {'size': 15}}, {'image': '../datasets/artificial_data/3d_shapes/volume_37_label_16.pt', 'label': {'size': 16}}, {'image': '../datasets/artificial_data/3d_shapes/volume_38_label_18.pt', 'label': {'size': 18}}, {'image': '../datasets/artificial_data/3d_shapes/volume_6_label_17.pt', 'label': {'size': 17}}, {'image': '../datasets/artificial_data/3d_shapes/volume_9_label_18.pt', 'label': {'size': 18}}, {'image': '../datasets/artificial_data/3d_shapes/volume_32_label_22.pt', 'label': {'size': 22}}, {'image': '../datasets/artificia

In [3]:
from src.transformers.TensorLoaderTransfomer import TensorLoader
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstD, ScaleIntensityd, ToTensord, ResizeD, Lambdad, NormalizeIntensityD

from src.transformers.SliceTransformer import SliceAggregateTransform
importlib.reload(sys.modules['src.transformers.SliceTransformer'])

importlib.reload(sys.modules['src.transformers.TensorLoaderTransfomer'])
transforms = [
    TensorLoader(keys=["image"]),
    ScaleIntensityd(keys=["image"], minv=0.0, maxv=1.0, factor=1.0),
    NormalizeIntensityD(keys=["image"], nonzero=True, channel_wise=True),
    ]

/Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <E03EDA44-89AE-3115-9796-62BA9E0E2EDE> /Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <D2077E4D-18BC-34B9-8A9B-1EF634A0F416> /Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
from src.dataLoaders.NiftiDataLoader2 import NiftiDataLoader
importlib.reload(sys.modules['src.dataLoaders.NiftiDataLoader2'])

<module 'src.dataLoaders.NiftiDataLoader2' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/NiftiDataLoader2.py'>

In [5]:
dataset = 'niftiShort'
data_path = '../datasets/nifti/'
data_dir = f'{data_path}{dataset}'

# data_dir = 'C:/Users/askel/Downloads/NIFTY/NIFTY/'
# max = 8876
dim = 384
data_loader = NiftiDataLoader(data_dir=data_dir, 
                              batch_size=32,
                              spatial_resize=(dim, dim, dim),
                              cache_rate=0.5, 
                              replace_rate=1,
                              transforms=transforms,
                              data_list=pairs,
                              )
# data_loader.load_data(visit_no=visit, subset_size=2058, cache="standard")
# data_loader.load_data(subset_size=84, cache="standard")
#data_loader.load_data(subset_size=4400, cache="standard")
# data_loader.load_data(subset_size=8875, cache="standard")

data_loader.load_data(cache="standard")

File /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/saved_data_lists/data_list.pkl does not exist.
Data list loaded: False
Using custom sampler: False
Subset size: 50


Loading dataset:   0%|          | 0/40 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 5/5 [00:01<00:00,  4.32it/s]

batch_size: 32


In [6]:
from src.models.medical_models.monai_resnet import MonaiMedicalResNet
importlib.reload(sys.modules['src.models.medical_models.monai_resnet'])
importlib.reload(sys.modules['src.models.medical_models.base_medical'])

<module 'src.models.medical_models.base_medical' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/base_medical.py'>

In [8]:
fusion_model = MonaiMedicalResNet(
    num_epochs=100,
    learning_rate=0.001,
    weight_decay=1e-4,
    data_loader=data_loader,
    dropout_rate=0.1,
    depth=18,
    pretrained=False
)

fusion_model.train()
fusion_model.evaluate()

Data loader train loader:  2 40
Image spatial dimensions:  torch.Size([32, 256, 256, 256])
Number of input channels:  256
Spatial dims:  2
gpu:  cpu
Is cuda available:  False cpu
Number of training images: 40
